# Project Luther

## Looking at Steam data to predict several factors on user rating


In [ ]:
#The basics
import numpy as np
import pandas as pd

#Get them web sites
import requests

#Make sure slenium works
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

#Start the google driver
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

#make sure to get a consistent table
import pickle
#need to pick apart strings
import re
import patsy

#needed for graphs
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

#For running regressions
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression

#Sklearn tools
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import matplotlib
matplotlib.style.use('fivethirtyeight') 


## Selenium is used to scrape data from steamspy.

In [ ]:
#Chrome driver is used to load Steamspy
driver = webdriver.Chrome(chromedriver)
driver.get('https://steamspy.com')

In [ ]:
#scrapes four pages of steamspy trending games for tables
#Data is scrapped on 4/21/2018
f = []
for i in range(4):
    i = driver.find_element_by_id('trendinggames')
    my_df = pd.read_html(driver.page_source)[0]
    i = i.text
    i = i.splitlines()
    i = i[5:]
    f.append(my_df)
    site = driver.find_element_by_xpath('//*[@id="trendinggames_next"]/a')
    site.click()
    time.sleep(1)
    

In [ ]:
#sanity check
pd.read_html(driver.page_source)[0]

In [ ]:
#Stacks the data and corrects the index issues
games_data = pd.concat(f)
games_data = games_data.reset_index()

In [ ]:
#pickles the data for later
games_data.to_pickle('/Users/NickThomas/Project_Luther/data/games_data.pkl')

## Running EDA on the pickled data

In [ ]:
#pickle sucessfully loaded
with open('/Users/NickThomas/Project_Luther/data/games_data.pkl', 'rb') as pickle_file:
        games_data_loaded = pickle.load(pickle_file)
games_data_loaded

In [ ]:
# provides basic info on the dataframe
games_data_loaded.info()

In [ ]:
#The value counts shows a high number of missing values for the user rating in the steam spy table
games_data_loaded['Score rank(Userscore / Metascore)'].value_counts()

## Additional data is gathered from Steam

Due to the complexity of the steam website and the fact that the data frame has 100 cases, searches for games were conducted manually while code was used to scrape each steam entry. Pages were scrapped on 4/22/2018. 

In [ ]:
# A games list is created for reference when making searches
game_list = [games_data_loaded.Game.unique()]
game_list

In [ ]:
# Chrome driver is loaded with the first entry in the list
driver = webdriver.Chrome(chromedriver)
driver.get('http://store.steampowered.com/search')
search = driver.find_element_by_xpath('//*[@id="term"]')
search.click()
search.send_keys("Friday the 13th: Killer Puzzle")
search.send_keys(Keys.RETURN)

In [ ]:
# blank lists are created for incoming variables
list3 = []
list6 = []
list7 = []

In [ ]:
# Elements are found by their XPATHs. Regular expresions are used to split a review string
#into user reviews as a percentage and the amount of people who leave reviews.
#Back up code is created if the game has no downloadable content(DLC)
rating = driver.find_element_by_xpath('//*[@id="game_highlights"]/div[1]/div/div[3]/div/div[1]')
a = rating.text
cost_dlc = driver.find_element_by_xpath('//*[@id="dlc_purchase_action"]/div[1]')
c = cost_dlc.text
#c = np.nan
f = re.compile(r'\d+\%')
g = re.findall(f, str(a))
h = re.compile(r'(?<=the\s).*(?=\suser)')
j = re.findall(h, str(a))
print(a)
print(c)
print(g)
print(j)

Note: Slight changes to the code were made depending on the circumstance. For example, c = np.nan is hashed out since I was normally expecting dlc cost. When a game didn't have dlc cost, the nan was put in and I hashed the other two out.

In [ ]:
# Information from each page is appended to a list.
list3.append(c)
list6.append(g)
list7.append(j)
print(list3)
print(list6)
print(list7)

In [ ]:
# Lists are converted to arrays
dlc_cost = np.asarray(list3)
user_rating = np.asarray(list6)
number_of_reviews = np.asarray(list7)

In [ ]:
#A dataframe is created form the arrays
form = {'dlc_cost': dlc_cost, 'user_rating': user_rating, 'number_of_reviews': number_of_reviews}
add_df = pd.DataFrame(form)
add_df.head()

In [ ]:
#The two dataframes are joined together
Steam_games_df = games_data_loaded.join(add_df)
Steam_games_df.head()

In [ ]:
#In case you need to jump in at this point
#quickstart
with open('/Users/NickThomas/Project_Luther/data/Steam_games_df.pkl', 'rb') as pickle_file:
        Steam_games_df = pickle.load(pickle_file)
Steam_games_df

In [ ]:
#Info is provided on the main data frame
Steam_games_df.info()

In [ ]:
#Values of nan are replaced with $0 to indicate that the game has no dlc or it was free
Steam_games_df['dlc_cost'] = Steam_games_df['dlc_cost'].replace('nan', '$0')

In [ ]:
#Null Values are dropped from the analysis
mask = Steam_games_df.user_rating.notnull()
Steam_games_df = Steam_games_df[mask]
Steam_games_df

In [ ]:
#This gets rid of lists created by scrapping data
Steam_games_df.reset_index()
for x in range(0,86):
    Steam_games_df.user_rating.iloc[x] = ''.join(Steam_games_df.user_rating.iloc[x])
for x in range(0,86):
    Steam_games_df.number_of_reviews.iloc[x] = ''.join(Steam_games_df.number_of_reviews.iloc[x])

In [ ]:
#Takes out the percentage sign in user_rating
for x in range(0,86):
    Steam_games_df.user_rating.iloc[x] = int(Steam_games_df.user_rating.iloc[x][:-1])

In [ ]:
#Takes out the $ sign in columns with price
Steam_games_df['Price'] = Steam_games_df['Price'].replace('Free', '$0')
for x in range(0,86):
    Steam_games_df.dlc_cost.iloc[x] = float(Steam_games_df.dlc_cost.iloc[x][1:])
for x in range(0,86):
    Steam_games_df.Price.iloc[x] = float(Steam_games_df.Price.iloc[x][1:])

In [ ]:
#Converts columns to floats and ints
Steam_games_df['Price'] = Steam_games_df.Price.astype(float)
Steam_games_df['dlc_cost'] = Steam_games_df.dlc_cost.astype(float)
Steam_games_df.number_of_reviews = Steam_games_df.number_of_reviews.str.replace(",", "")
Steam_games_df['number_of_reviews'] = Steam_games_df.number_of_reviews.astype(int)

In [ ]:
#Converts release date to time series
Steam_games_df['Release_date'] = Steam_games_df['Release date']
for x in range(0,86):
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 27, 2018', 'March 27, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 26, 2018', 'March 26, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 28, 2018', 'March 28, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 23, 2018', 'March 23, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 30, 2018', 'March 30, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 22, 2018', 'March 22, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 13, 2018', 'March 13, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 20, 2018', 'March 20, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 9, 2018', 'March 9, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 29, 2018', 'March 29, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 21, 2018', 'March 21, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 19, 2018', 'March 19, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 15, 2018', 'March 15, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 14, 2018', 'March 14, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 12, 2018', 'March 12, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 25, 2018', 'March 25, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Mar 8, 2018', 'March 8, 2018').iloc[x]
for x in range(0,86):
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 12, 2018', 'April 12, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 10, 2018', 'April 10, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 6, 2018', 'April 6, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 19, 2018', 'April 19, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 3, 2018', 'April 3, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 9, 2018', 'April 9, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 11, 2018', 'April 11, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 5, 2018', 'April 5, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 18, 2018', 'April 18, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 13, 2018', 'April 13, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 16, 2018', 'April 16, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 14, 2018', 'April 14, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 2, 2018', 'April 2, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 7, 2018', 'April 7, 2018').iloc[x]
    Steam_games_df['Release_date'].iloc[x] = Steam_games_df['Release_date'].replace('Apr 4, 2018', 'April 4, 2018').iloc[x]
Steam_games_df['Release_date'] = pd.to_datetime(Steam_games_df.Release_date)


In [ ]:
#Creates dummy variables for category
Owner_dummy2 = patsy.dmatrix('Owners',data=Steam_games_df,return_type='dataframe')
Owner_dummy2.columns = column_names
Steam_games_dumb_df = Steam_games_df.join(Owner_dummy2)
Steam_games_dumb_df

In [ ]:
#deletes unnecessary columns in the dataframe
delete = ["index", "Score rank(Userscore / Metascore", "#", "Release date"]
Steam_games_dumb_df = Steam_games_dumb_df.drop(columns=["index", "Score rank(Userscore / Metascore)", "#", "Release date"])

In [ ]:
#assigns the owners column as a categorical variable
Steam_games_dumb_df['Owners'] = Steam_games_dumb_df.Owners.astype('category').cat.codes
Steam_games_dumb_df.assign(Owners=Steam_games_dumb_df.Owners.astype('category').cat.codes).corr()

I decided not to chase outliers as I was given advice that in sales data outliers can be important. When looking at variables such as user rating I also wanted to have some idea for where games hit low ratings even if there are not that many of them.

## Building my model

In [ ]:
#Prints correlation table of the variables
Steam_games_dumb_df.corr()['user_rating'].plot.barh(color='red', edgecolor='black')
print(Steam_games_dumb_df.corr()['user_rating'].sort_values())
plt.xlabel("Correlation")
plt.title("Correlations Table")
plt.show

In [ ]:
#Create a test-train split
x_list = ['Price', 'Owners']
X = Steam_games_dumb_df[x_list]
y = Steam_games_dumb_df.user_rating
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

In [ ]:
#Run a grid search with lasso regression
model = Lasso(max_iter=5000)
parameters = {'alpha': [1e-5,1e-3,1e-1,1], 'fit_intercept': [True,False]}
grid = GridSearchCV(model,parameters, cv=5, scoring='neg_mean_absolute_error', n_jobs=1)
grid.fit(X, y)

In [ ]:
grid.cv_results_

In [ ]:
#get a look at the predicted and residual values
best_lasso = grid.best_estimator_
lasso_pred = best_lasso.predict(X_test)
for true,pred in zip(y_test[:20], lasso_pred[:20]):
    resid = true - pred
    print("pred, resid:", str(pred) + ", "+ str(resid))

In [ ]:
#Look at the parameters
print(grid.best_params_, grid.best_score_)

In [ ]:
#Running a Lasso regression
lr2 = Lasso(alpha = .00001)
lr2.fit(X_train, y_train)
score2 = lr2.score(X_test, y_test)
print("Lasso Regression: ", score2)
print("Coef:", lr2.coef_)
y_pred = lr2.predict(X_test)
print("MSE test:", mean_squared_error(y_test, y_pred))

In [ ]:
#Create a plot of actual versus predicted values
plt.scatter(y_pred, y_test)
plt.axhline(y=0, color='k')
plt.plot([55,80],[0,100], 'k--')
plt.grid()
plt.xlabel("Predicted User Rating")
plt.ylabel("Actual User Rating")
plt.title("Actual vs Predicted Values")
plt.show


In [ ]:
#Plot the residuals
plt.figure(figsize=(12,8))
plt.scatter(y_pred, y_pred - y_test, c='blue', alpha=0.1, s=100, label='(Testing Residuals)')

plt.title("Residual Plot")
plt.ylim(-10,10)

It looks like this model has no predictive power. I decided to search for more data in hope of finding other features that could possibily predict user rating in a different dataset.

## Building a new model with kaggle data

In [ ]:
#Load in Kaggle dataset
Steam_kaggle = pd.read_csv('/Users/NickThomas/Project_Luther/data/Video_Game_Sales_as_of_Jan_2017.csv')

In [ ]:
#get info on new dataset
Steam_kaggle.info()

In [ ]:
#Drop null values
Steam_kaggle['User_Score'] = Steam_kaggle.User_Score.replace('tbd', np.NaN)
Steam_kaggle = Steam_kaggle[Steam_kaggle.User_Score.notnull()]
Steam_kaggle = Steam_kaggle[Steam_kaggle.Critic_Score.notnull()]
Steam_kaggle_nonnull.info()
Steam_kaggle_nonull = Steam_kaggle.dropna()
Steam_kaggle_nonnull.info()


In [ ]:
#Convert user score to float
Steam_kaggle['User_Score'] = Steam_kaggle.User_Score.astype(float)

In [ ]:
#Create a test-train split
x_list = ['Critic_Score', 'Critic_Count', 'Year_of_Release', 'Global_Sales']
X = Steam_kaggle_nonull[x_list]
y = Steam_kaggle_nonull.User_Score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
#Run a grid search on Lassor regression for the parameters
model = Lasso(max_iter=5000)
parameters = {'alpha': [1e-5,1e-3,1e-1,1], 'fit_intercept': [True,False]}
grid = GridSearchCV(model,parameters, cv=5, scoring='neg_mean_absolute_error', n_jobs=1)
grid.fit(X, y)

In [ ]:
grid.cv_results_

In [ ]:
#Get an idea of predicted values and residuals
best_lasso = grid.best_estimator_
lasso_pred = best_lasso.predict(X_test)
for true,pred in zip(y_test[:20], lasso_pred[:20]):
    resid = true - pred
    print("pred, resid:", str(pred) + ", "+ str(resid))

In [ ]:
#parameters for the Lassor regression
print(grid.best_params_, grid.best_score_)

In [ ]:
#Running a Lasso regression
lr2 = Lasso(alpha = 0.1)
lr2.fit(X_train, y_train)
score2 = lr2.score(X_test, y_test)
print("Lasso Regression: ", score2)

y_pred = lr2.predict(X_test)
print("Coef:", lr2.coef_)
print("MSE test:", mean_squared_error(y_test, y_pred))


In [ ]:
#Plot the predicted and actual values for user rating
plt.scatter(y_pred, y_test,alpha=0.1)
plt.axhline(y=0, color='k')
plt.plot([3,10],[3,10], 'k--')
plt.grid()
plt.xlabel("Predicted User Rating")
plt.ylabel("Actual User Rating")
plt.title("Actual vs Predicted Values")
plt.show()

In [ ]:
#Plot the residuals
plt.figure(figsize=(12,8))
plt.scatter(y_pred, y_pred - y_test, c='blue', alpha=0.1, s=100, label='(Testing Residuals)')

plt.title("Residual Plot")
plt.ylim(-10,10)